<a href="https://colab.research.google.com/github/armandpriko/appdev/blob/main/Microsoft_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")


In [ ]:
test = pd.read_csv("/kaggle/input/zindinlp/Test.csv")
train = pd.read_csv("/kaggle/input/zindinlp/Train.csv")

In [ ]:
train.info()

#### we have some missing text and location
the missing locations are going to become a test file and the missing text are going to be dropped cuz we don't have any informations from them

In [ ]:
trainclean = train.dropna(subset=['text'])

In [ ]:
testsup = trainclean[trainclean["location"].isna()]

In [ ]:
testsup

In [ ]:
print(train.shape)
print(trainclean.shape)
print(test.shape)
print(testsup.shape)

In [ ]:
trainclean.dropna(inplace = True)

In [ ]:
print(train.shape)
print(trainclean.shape)
print(test.shape)
print(testsup.shape)

In [ ]:
trainclean.reset_index(drop=True,inplace = True)
testsup.reset_index(drop=True,inplace = True)
train.reset_index(drop=True,inplace = True)

## This is the first idea that comes to mind!:

In [ ]:
locations = trainclean.location.unique()
locations.tolist()

In [ ]:
import spacy
import pandas as pd
from textblob import TextBlob
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


In [ ]:
def extract_locations(text, location_list):
    doc = nlp(text)
    locations_found = []
    for ent in doc.ents:
        if ent.label_ == 'GPE' and ent.text in location_list:
            locations_found.append(ent.text)
    return locations_found

In [ ]:
testsup['location'] = testsup['text'].apply(lambda x: extract_locations(x, locations))

In [ ]:
testsup['location'] = testsup['location'].astype(str)
testsup['location'] = testsup['location'].str.strip('[]').str.strip()
testsup['location'] = testsup['location'].str.replace("'", "")
testsup['location'] = testsup['location'].str.replace(",", "")
testsup['location'] = testsup['location'].replace(r'^\s*$', '', regex=True)
testsup

### Let's add to the train data

In [ ]:
testsup.drop('location',axis=1,inplace=True)

In [ ]:
"""
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load Spacy model
nlp = spacy.load('en_core_web_sm')

# Function to extract locations
def extract_locations(text):
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    return ' '.join(locations)

# Apply the function to the training data
trainclean['extracted_locations'] = trainclean['text'].apply(extract_locations)

# Initialize TfidfVectorizer and fit it on the training data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(trainclean['extracted_locations'])

# Assume 'location' is your target column
X_train, X_test, y_train, y_test = train_test_split(X_train, trainclean['location'], test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict on the test set and print classification report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Apply the function to the test data
testsup['extracted_locations'] = testsup['text'].apply(extract_locations)

# Transform the test data using the same vectorizer
X_test_transformed = vectorizer.transform(testsup['extracted_locations'])

# Predict on the transformed test data
y_pred = model.predict(X_test_transformed)

# Save the predictions to a CSV file
import pandas as pd

submission = pd.DataFrame({
    'tweet_id': testsup['tweet_id'],  # Replace with your actual ID column
    'location': y_pred
})"""

In [ ]:
# submission2 = pd.read_csv("/kaggle/working/submission.csv")

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForTokenClassification, pipeline

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')

# Create NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

def extract_locations(text):
    ner_results = nlp(text)
    locations = [result['word'] for result in ner_results if result['entity'] == 'B-LOC' or result['entity'] == 'I-LOC']
    return ' '.join(locations)
from sklearn.model_selection import train_test_split



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(trainclean['text'], trainclean['location'], test_size=0.2, random_state=42)
# Apply the function to the text column
X_train_extracted = X_train.apply(lambda x: extract_locations(x) if isinstance(x, str) else '')
X_test_extracted = X_test.apply(lambda x: extract_locations(x) if isinstance(x, str) else '')
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_extracted)
X_test_vectorized = vectorizer.transform(X_test_extracted)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Train a RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_vectorized)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(classification_report(y_test, y_pred))


In [ ]:
submission = pd.DataFrame({
    'tweet_id': test['tweet_id'],  # Replace with your actual ID column
    'location': test["extracted_locations"]
})
submission.to_csv('submission2.csv', index=False)

### let's handle the Hashtags and @s

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, pipeline
from sklearn.metrics import accuracy_score

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model_ner = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english').to('cuda')

# Create NER pipeline
nlp = pipeline("ner", model=model_ner, tokenizer=tokenizer, device=0)

def preprocess_text(text):
    """Replace @ with 'at' and extract hashtags."""
    text = text.replace('@', ' at ')
    hashtags = re.findall(r'#(\w+)', text)
    text = re.sub(r'#\w+', '', text)
    hashtag_locations = ' '.join(hashtags)
    return text, hashtag_locations

def extract_locations(text):
    """Extract locations using BERT NER."""
    ner_results = nlp(text)
    locations = [result['word'] for result in ner_results if result['entity'] == 'B-LOC' or result['entity'] == 'I-LOC']
    return ' '.join(locations)

# Preprocess text in both DataFrames
trainclean['text'], trainclean['hashtag_locations'] = zip(*trainclean['text'].apply(preprocess_text))
testsup['text'], testsup['hashtag_locations'] = zip(*testsup['text'].apply(preprocess_text))

# Extract locations from the text
trainclean['extracted_locations'] = trainclean['text'].apply(lambda x: extract_locations(x) if isinstance(x, str) else '')
testsup['extracted_locations'] = testsup['text'].apply(lambda x: extract_locations(x) if isinstance(x, str) else '')

# Combine extracted locations and hashtag locations
trainclean['final_extracted_locations'] = trainclean[['extracted_locations', 'hashtag_locations']].apply(lambda x: ' '.join(filter(None, x)), axis=1)
testsup['final_extracted_locations'] = testsup[['extracted_locations', 'hashtag_locations']].apply(lambda x: ' '.join(filter(None, x)), axis=1)

# Vectorize the final extracted locations
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(trainclean['final_extracted_locations'])
X_test_vectorized = vectorizer.transform(testsup['final_extracted_locations'])

# Train-Test Split for training data
X_train, X_val, y_train, y_val = train_test_split(X_train_vectorized, trainclean['location'], test_size=0.2, random_state=42)

# Train the RandomForest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = rf_model.predict(X_val)
print(f'Validation Accuracy: {accuracy_score(y_val, y_val_pred)}')

# Predict on test data
testsup['predicted_location'] = rf_model.predict(X_test_vectorized)

# Display results
testsup


# Nebraska doesn't seem like the correct prediction

In [ ]:
add = testsup[testsup['predicted_location']!='Nebraska']

In [ ]:
add

In [ ]:
supp = pd.DataFrame({
    'tweet_id': add['tweet_id'],  # Replace with your actual ID column
    'text' : add["text"],
    'location': add[add["predicted_location"]!='Nebraska'].predicted_location
})
supp.dropna(inplace = True)

In [ ]:
print(supp.shape)
print(trainclean.shape)

In [ ]:
del X_test_vectorized,X_train_vectorized,X_train, X_val, y_train, y_val,add,testsup

## now we redifine trainclean to get 3 cols

In [ ]:
print(supp.shape)
print(trainclean.shape)
supp.reset_index(inplace = True)

In [ ]:
trainclean = pd.concat([trainclean,supp])
trainclean

In [ ]:
trainclean.drop("index",axis = 1,inplace = True)

In [ ]:
trainclean.reset_index(inplace = True)
trainclean

In [ ]:
#del supp

# now we train the model with the new DF

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, pipeline
from sklearn.metrics import accuracy_score

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model_ner = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english').to('cuda')

# Create NER pipeline
nlp = pipeline("ner", model=model_ner, tokenizer=tokenizer, device=0)

def preprocess_text(text):
    """Replace @ with 'at' and extract hashtags."""
    text = text.replace('@', ' at ')
    hashtags = re.findall(r'#(\w+)', text)
    text = re.sub(r'#\w+', '', text)
    hashtag_locations = ' '.join(hashtags)
    return text, hashtag_locations

def extract_locations(text):
    """Extract locations using BERT NER and custom rules."""
    # Custom rule-based extraction for hashtags and common indicators
    text, hashtag_locations = preprocess_text(text)
    common_indicators = ['Municipality', 'City', 'Town', 'Village']
    additional_locations = ' '.join([word for word in text.split() if any(ind in word for ind in common_indicators)])

    ner_results = nlp(text)
    locations = [result['word'] for result in ner_results if result['entity'] == 'B-LOC' or result['entity'] == 'I-LOC']

    # Combine NER and custom rule-based locations
    all_locations = ' '.join(locations + additional_locations.split() + hashtag_locations.split())
    return all_locations

# Preprocess text in both DataFrames
trainclean['text'], trainclean['hashtag_locations'] = zip(*trainclean['text'].apply(preprocess_text))
test['text'], test['hashtag_locations'] = zip(*test['text'].apply(preprocess_text))

# Extract locations from the text
trainclean['extracted_locations'] = trainclean['text'].apply(lambda x: extract_locations(x) if isinstance(x, str) else '')
test['extracted_locations'] = test['text'].apply(lambda x: extract_locations(x) if isinstance(x, str) else '')

# Combine extracted locations and hashtag locations
trainclean['final_extracted_locations'] = trainclean[['extracted_locations', 'hashtag_locations']].apply(lambda x: ' '.join(filter(None, x)), axis=1)
test['final_extracted_locations'] = test[['extracted_locations', 'hashtag_locations']].apply(lambda x: ' '.join(filter(None, x)), axis=1)

# Vectorize the final extracted locations
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(trainclean['final_extracted_locations'])
X_test_vectorized = vectorizer.transform(test['final_extracted_locations'])

# Train-Test Split for training data
X_train, X_val, y_train, y_val = train_test_split(X_train_vectorized, trainclean['location'], test_size=0.2, random_state=42)

# Train the RandomForest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = rf_model.predict(X_val)
print(f'Validation Accuracy: {accuracy_score(y_val, y_val_pred)}')

# Predict on test data
test['predicted_location'] = rf_model.predict(X_test_vectorized)

# Display results
print(test[['text', 'final_extracted_locations', 'predicted_location']])


## Dealing with some Submissions

In [ ]:
submission = pd.DataFrame({
    'tweet_id': test['tweet_id'],  # Replace with your actual ID column
    'location': test["predicted_location"]
})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission3 = pd.read_csv("/kaggle/input/submissions/submission3.csv")
submission2 = pd.read_csv("/kaggle/input/submissions/submission2.csv")
submission1 = pd.read_csv("/kaggle/input/submissions/submission1.csv")

In [ ]:
missing_sub3 = (submission3[submission3["location"].isna()].tweet_id).tolist()

In [ ]:
missing_sub2 = (submission2[submission2["location"].isna()].tweet_id).tolist()

In [ ]:
set3 = set(missing_sub3)
set2 = set(missing_sub2)
missing_all = set2 | set3

In [ ]:
testmissing = test[test['tweet_id'].isin(missing_all)]

In [ ]:
testmissing.reset_index(drop=True,inplace = True)
testmissing

In [ ]:
testmissing['text'] = testmissing['text'].str.replace('#', ' ')
testmissing['text'] = testmissing['text'].str.replace('@', 'at ')
testmissing

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, pipeline
from sklearn.metrics import accuracy_score

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model_ner = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english').to('cuda')

# Create NER pipeline
nlp = pipeline("ner", model=model_ner, tokenizer=tokenizer, device=0)

def preprocess_text(text):
    """Replace @ with 'at', extract hashtags, and clean text."""
    text = text.replace('@', ' at ')
    hashtags = re.findall(r'#(\w+)', text)
    text = re.sub(r'#\w+', '', text)
    hashtag_locations = ' '.join(hashtags)
    return text, hashtag_locations

def extract_locations_and_more(text):
    """Extract locations, roads, streets, localities, districts, and countries."""
    # Custom rule-based extraction for roads, streets, localities, etc.
    road_patterns = re.findall(r'\b(\d+)\b', text)  # Extract road numbers
    street_patterns = re.findall(r'\b(?:street|st|road|rd|avenue|ave|blvd)\b', text, re.IGNORECASE)
    locality_patterns = re.findall(r'\b(?:locality|district)\b', text, re.IGNORECASE)
    country_patterns = re.findall(r'\b(?:country|nation)\b', text, re.IGNORECASE)

    # Clean text
    text, hashtag_locations = preprocess_text(text)

    # Extract using BERT NER
    ner_results = nlp(text)
    locations = [result['word'] for result in ner_results if result['entity'] == 'B-LOC' or result['entity'] == 'I-LOC']

    # Combine all extracted information
    all_locations = ' '.join(locations + road_patterns + street_patterns + locality_patterns + country_patterns + hashtag_locations.split())
    return all_locations

# Preprocess text in both DataFrames
trainclean['text'], trainclean['hashtag_locations'] = zip(*trainclean['text'].apply(preprocess_text))
testmissing['text'], testmissing['hashtag_locations'] = zip(*testmissing['text'].apply(preprocess_text))

# Extract locations and additional information from the text
trainclean['extracted_info'] = trainclean['text'].apply(lambda x: extract_locations_and_more(x) if isinstance(x, str) else '')
testmissing['extracted_info'] = testmissing['text'].apply(lambda x: extract_locations_and_more(x) if isinstance(x, str) else '')

# Vectorize the extracted information
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(trainclean['extracted_info'])
X_test_vectorized = vectorizer.transform(testmissing['extracted_info'])

# Train-Test Split for training data
X_train, X_val, y_train, y_val = train_test_split(X_train_vectorized, trainclean['location'], test_size=0.2, random_state=42)

# Train the RandomForest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = rf_model.predict(X_val)
print(f'Validation Accuracy: {accuracy_score(y_val, y_val_pred)}')

# Predict on test data
testmissing['predicted_location'] = rf_model.predict(X_test_vectorized)

# Display results
print(testmissing[['text', 'extracted_info', 'predicted_location']])


In [ ]:
submission = pd.DataFrame({
    'tweet_id': testmissing['tweet_id'],  # Replace with your actual ID column
    'location': testmissing["predicted_location"]
})
submission.to_csv('submission22.csv', index=False)

In [ ]:
submissing1 = pd.read_csv("/kaggle/working/submissionmissing.csv")
submissing2 = pd.read_csv("/kaggle/working/submission22.csv")

In [ ]:
submission = pd.DataFrame({
    'tweet_id': result1['tweet_id'],  # Replace with your actual ID column
    'location': result1["result"]
})
submission.to_csv('subxx.csv', index=False)